In [1]:
import pymysql
import pandas as pd
import numpy as np
import time
import datetime
import os
from sqlalchemy import create_engine  

In [2]:
db_info = {'user':'bo.ma',  
            'password':'123456',  
            'host':'192.168.100.141',  
            'database':'siyecap'  # 这里我们事先指定了数据库，后续操作只需要表即可
            } 

# 进行查询的语句
engine = create_engine('mysql+pymysql://%(user)s:%(password)s@%(host)s/%(database)s?charset=utf8' % db_info,encoding='utf-8') 
#这里直接使用pymysql连接,echo=True，会显示在加载数据库所执行的SQL语句。

In [3]:
data_path = r'F:\bma\project\data'
startdate = '2007-01-01'
enddate = '2019-12-01'
data_name = 'open,close,high,low,tradingstatus,volume,turnoverrate,volumeyuan'
sql = '''SELECT symbol,date,%s FROM siyecap.stockprice where date between '%s' and '%s';'''%(data_name,startdate,enddate)

## 提取行情数据，整合、处理

In [ ]:
%%time
# 提取一个字段大概需要2min 批量提取字段可以减少总时间
data = pd.read_sql_query(sql,con = engine)

In [ ]:
data = data.sort_values(by = ['date','symbol'])
data.index = range(len(data.index))

In [ ]:
def get_tradedays():
    # 返回的为 dataframe 时间为2007/01/01 - 2019/12/01 
    tradeday = pd.read_csv(os.path.join(data_path,'tradeday.csv'),parse_dates = ['date'])
    
    return tradeday

def get_tickers():
    tickers = pd.read_csv(os.path.join(data_path,'tickers.csv'))
    print('there are %s stocks' % len(tickers))
    return tickers

In [ ]:
tradeday = get_tradedays()
tradeday.set_index(['date'],inplace = True,drop=False)

In [ ]:
widedata = data.pivot_table(columns = ['symbol'], index= ['date'],values=['open'],dropna= False)
widedata.columns = widedata.columns.droplevel()
# set(widedata.columns)- set(data['symbol'])

In [ ]:
def save_data(data_name_list,startdate =  '2007-01-01',enddate = '2019-12-01',engine = engine): 
    data_str = ','.join(data_name_list)
    sql = '''SELECT symbol,date,%s FROM siyecap.stockprice where date between '%s' and '%s';'''%(data_str,startdate,enddate)
    time1 = time.time()
    data = pd.read_sql_query(sql,con = engine)
    data = data.sort_values(by = ['date','symbol'])
    data.index = range(len(data.index))
    for data_name in data_name_list:
        time10 = time.time()
        widedata = data[['symbol','date',data_name]].pivot_table(columns = ['symbol'], index= ['date'],values=[data_name],dropna= False)
        widedata.columns = widedata.columns.droplevel()
        widedata.to_csv(os.path.join(data_path,'%s.csv')%data_name)
        time11 = time.time()
        print('getting and reshaping %s data cost %.2f s'%(data_name,time11-time10))
    time2 = time.time()
    print('getting and reshaping all data cost %.2f s'%(time2-time1))

In [ ]:
%%time
data_name_list = ['tradingstatus','close','open','high','low','avgprice','volume','adjustfactor','volumeyuan','turnoverrate',
                  'pe','pe_ttm','ps','ps_ttm','pb','est_eps']
save_data(data_name_list)

In [ ]:
test_close =  pd.read_csv(os.path.join(data_path,'close'+'.csv'),index_col= 0)

In [ ]:
close = pd.read_csv(os.path.join(data_path,'close.csv'),index_col=0)
adjustfactor = pd.read_csv(os.path.join(data_path,'adjustfactor.csv'),index_col=0)
close_re = close*adjustfactor

In [ ]:
# 将原始数据处理后复权的数据 可以进行前后对比
data_path = r'F:\bma\project\data'
data_list = ['close', 'open', 'high', 'low']

data_dict = {}
adjustfactor = pd.read_csv(os.path.join(data_path,'adjustfactor.csv'),index_col=0)
for data_name in data_list:
    tempdata = pd.read_csv(os.path.join(data_path,data_name+'_raw.csv'),index_col=0)
    tempdata = tempdata * adjustfactor
    tempdata.to_csv(os.path.join(data_path,data_name+'.csv'))

提取指数数据

In [ ]:
sql = '''SELECT symbol,date,%s FROM siyecap.indexprice where date between '%s' and '%s';'''%(data_name,startdate,enddate)
data = pd.read_sql_query(sql,con = engine)

In [ ]:
data = data.sort_values(by = ['date','symbol'])
data.index = range(len(data.index))
data.set_index(['symbol','date'],inplace=True)
data.reset_index(inplace=True)

In [ ]:
index_name = ['000001.SH','000300.SH','399001.SH','000016.SH']

In [ ]:
data_name = 'close'
time10 = time.time()
widedata = data[['symbol','date',data_name]].pivot_table(columns = ['symbol'], index= ['date'],values=[data_name],dropna= False)
widedata.columns = widedata.columns.droplevel()
# widedata.to_csv(os.path.join(data_path,'%s.csv')%data_name)
time11 = time.time()

In [ ]:
widedata.to_csv(os.path.join(data_path,'index_%s.csv')%data_name)

获取涨跌停以及停牌数据

In [19]:
sql = '''SELECT symbol,date,%s FROM siyecap.stockinfo where date = '2019-10-10';'''%('ipodays') # stockinfo 里面没有科创板
data = pd.read_sql_query(sql,con = engine)

In [4]:
# 提取科创板的list
tickers  = pd.read_csv(os.path.join(data_path,'tickers.csv'))

In [13]:
tickers_kechuang = [x for x in  tickers.values.T[0] if x[:3] == '688' ]

In [26]:
tickers_kechuang2 = [x for x in  list(set(tickers.values.T[0]) - set(tickers2.values.T[0])) if x[:3] == '688' ]   

In [28]:
len(tickers_kechuang2)

56